# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [71]:
!pip install --upgrade allennlp==0.9.0
!conda install -c conda-forge spacy
!python -m spacy download en_core_web_sm
import spacy
print(spacy.__version__)

Requirement already up-to-date: allennlp==0.9.0 in /usr/local/lib/python3.7/dist-packages (0.9.0)
/bin/bash: conda: command not found
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
2.1.0


In [72]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(1)

In [73]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [74]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

from itertools import chain
vocab = Vocabulary.from_instances(chain(train_dataset, validation_dataset))

562it [00:00, 12764.87it/s]
23it [00:00, 3749.86it/s]
585it [00:00, 35359.01it/s]


## Define and Train Model

In [75]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9074 ||: 100%|██████████| 281/281 [00:01<00:00, 146.68it/s]
accuracy: 0.7878, loss: 1.1925 ||: 100%|██████████| 12/12 [00:00<00:00, 301.29it/s]
accuracy: 0.8442, loss: 0.7301 ||: 100%|██████████| 281/281 [00:01<00:00, 162.20it/s]
accuracy: 0.7878, loss: 1.1922 ||: 100%|██████████| 12/12 [00:00<00:00, 302.51it/s]
accuracy: 0.8442, loss: 0.7165 ||: 100%|██████████| 281/281 [00:01<00:00, 159.84it/s]
accuracy: 0.7878, loss: 1.1849 ||: 100%|██████████| 12/12 [00:00<00:00, 293.90it/s]
accuracy: 0.8442, loss: 0.7069 ||: 100%|██████████| 281/281 [00:01<00:00, 161.01it/s]
accuracy: 0.7878, loss: 1.1666 ||: 100%|██████████| 12/12 [00:00<00:00, 292.19it/s]
accuracy: 0.8442, loss: 0.6992 ||: 100%|██████████| 281/281 [00:01<00:00, 160.59it/s]
accuracy: 0.7878, loss: 1.1706 ||: 100%|██████████| 12/12 [00:00<00:00, 320.61it/s]
accuracy: 0.8442, loss: 0.6911 ||: 100%|██████████| 281/281 [00:01<00:00, 161.34it/s]
accuracy: 0.7878, loss: 1.1577 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 98,
 'best_validation_accuracy': 0.8877551020408163,
 'best_validation_loss': 0.37295930882100947,
 'epoch': 99,
 'peak_cpu_memory_MB': 2020.296,
 'peak_gpu_0_memory_MB': 442,
 'training_accuracy': 0.9273382056985489,
 'training_cpu_memory_MB': 2020.296,
 'training_duration': '0:03:06.779838',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 442,
 'training_loss': 0.1863348558221967,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8795918367346939,
 'validation_loss': 0.3805924288947911}

## Evaluation

The simple code below loops over the validation set, applying the model to each exmaple and collecting out the input token, gold-standard output, and model output. You can see from these methods how to access ground truth and model outputs for evaluation.

In [76]:
def tag_sentence(s):
  tag_ids = np.argmax(model.forward_on_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
## Show the first example
baseline_output[0]

[(With, 'O', 'O'),
 (a, 'O', 'O'),
 (wave, 'O', 'O'),
 (of, 'O', 'O'),
 (his, 'O', 'O'),
 (hand, 'O', 'O'),
 (,, 'O', 'O'),
 (Peng, 'B-PERSON', 'B-PERSON'),
 (Dehuai, 'I-PERSON', 'I-PERSON'),
 (said, 'O', 'O'),
 (that, 'O', 'O'),
 (despite, 'O', 'O'),
 (being, 'O', 'O'),
 (over, 'O', 'O'),
 (100, 'B-CARDINAL', 'B-CARDINAL'),
 (regiments, 'O', 'O'),
 (,, 'O', 'O'),
 (let, 'O', 'O'),
 ('s, 'O', 'O'),
 (call, 'O', 'O'),
 (this, 'O', 'O'),
 (campaign, 'O', 'O'),
 (the, 'B-EVENT', 'O'),
 (Hundred, 'I-EVENT', 'I-EVENT'),
 (Regiments, 'I-EVENT', 'I-EVENT'),
 (Offensive, 'I-EVENT', 'I-EVENT'),
 (., 'O', 'O')]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [77]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for each in tagged:
    for i in range(len(each)):
      if each[i-1][2][:2] == 'O' and each[i][2][:2] == 'I-':
        count += 1
      if each[i-1][2][:2] == 'I-' and each[i][2][:2] == 'I-':
        if each[i-1][2][2:] != each[i][2][2:]:
          count += 1
      if each[i-1][2][:2] == 'B-' and each[i][2][:2] == 'I-':
        if each[i-1][2][2:] != each[i][2][2:]:
          count += 1
      if i == 0 and each[i][2][0:2] == 'I-':
        count+=1
  return count

# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# This is harsher than the token-level metric that the
# LSTM was trained to optimize, but it is the standard way
# of evaluating NER systems.
# Take tagger output as input

def check_list(li):
  span = []
  for tag in li:
    i = 0
    spans = []
    while i < len(tag):
      if tag[i][0] == "B":
        first, second = tag[i].split("-")
        j = i+1
        start = i
        end = i
        while j < len(tag):
          if tag[j][0] != "I":
            break
          else:
            if tag[j].split("-")[1] != second:
              break
            else:
              end = j
              j+=1
        spans.append((second, start, end))
        i = j
      else:
        i += 1
    span.append(spans)
  temp = 0
  for i in span:
    temp += len(i) 
  return span, temp


def span_stats(tagged):
  pred_li = [[word[2] for word in lst] for lst in tagged]
  gold_li = [[word[1] for word in lst] for lst in tagged]
  actual_span = []
  pred_span = []
  actual = 0
  pred = 0
  pred_span, pred = check_list(pred_li)
  actual_span, actual = check_list(gold_li)
  trueSpan = 0
  for i in range(len(pred_span)):
    for j in range(len(pred_span[i])):
      if pred_span[i][j] in actual_span[i]:
        trueSpan += 1
  precision = trueSpan/pred
  recall = trueSpan/actual
  f1 = (2 * precision * recall) / (precision + recall)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}

print(violations(baseline_output))
print(span_stats(baseline_output))

37
{'precision': 0.3448275862068966, 'recall': 0.23255813953488372, 'f1': 0.2777777777777778}


## Decoding

Now you can finally implement the simple Viterbi decoder. The `model` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the expression `model.forward_on_instance(s)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the model object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [78]:
# This code shows how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))

def viterbi_decoder(validation_dataset):
    predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
    labels = vocab.get_index_to_token_vocabulary('labels')

    unique_l = list(labels.values())
    for sentence in validation_dataset:
      emission_scores = predictor.predict_instance(sentence)['tag_logits']
      emissionProb = pd.DataFrame(emission_scores, index = [token.text for token in list(sentence['tokens'])], columns = unique_l)

    
    # Creating a K by K transition matrix, where K is the number of tags
    transitionMatrix = pd.DataFrame(index=labels.values(),columns=labels.values())
    transitionMatrix = transitionMatrix.fillna(0)  
    for tag_index_1 in range(len(transitionMatrix)):
      for tag_index_2 in range(len(transitionMatrix)):
        if unique_l[tag_index_2] != 'O':
          prev = unique_l[tag_index_1][0]
          curr = unique_l[tag_index_2][0]
          prevLabelType= unique_l[tag_index_1][2:]
          currLabelType= unique_l[tag_index_2][2:]
    
          #Rules to fill -inf to invalid steps
          #1 curr =='I' and prev=='O'
          if(curr == 'I' and prev == 'O'):  
            transitionMatrix.iloc[tag_index_1,tag_index_2] = np.log(0)
          #2 curr =='I' and prev=='I' and prevLabelType!= currLabelType
          elif(curr =='I' and prev == 'I' and prevLabelType != currLabelType): 
            transitionMatrix.iloc[tag_index_1,tag_index_2] = np.log(0)
          #3 curr =='I' and prev=='B' and prevLabelType!= currLabelType
          elif(curr == 'I' and prev == 'B' and prevLabelType != currLabelType): 
            transitionMatrix.iloc[tag_index_1,tag_index_2] = np.log(0)    
          else:
            continue  

    start_probs = [0.0] * len(unique_l)
    tags = dynamic_path(start_probs, emissionProb, transitionMatrix)
    tags = zip(sentence['tokens'], sentence['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tags])
    tags_cum = []
    tags_cum.append(list(tags))
    return tags_cum

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [79]:
def dynamic_path(start_probs, emissionProb, transitionProb):
    dp_table = np.array([[0.0 for j in range(emissionProb.shape[0])] for i in range(len(start_probs))])
    backtrack_table = np.array([[-1 for j in range(emissionProb.shape[0])] for i in range(len(start_probs))])
    
    for i in range(emissionProb.shape[0]):
        if i == 0:
            dp_table[:,0] = np.add(start_probs, emissionProb.iloc[i])
        else:
            for j in range(len(start_probs)):
                dp_table[j,i]= np.max(np.add(transitionProb.iloc[:,j] , dp_table[:,i-1])) + emissionProb.iloc[i,j]
                backtrack_table[j,i] = np.argmax(np.add(transitionProb.iloc[:,j] , dp_table[:,i-1]))
  
    score = np.max(dp_table[:,i])
    max_index = np.argmax(dp_table[:,i])
  
    final_li = []
    while max_index != -1:
      final_li.append(max_index)
      max_index = backtrack_table[max_index,i]
      i -= 1
    final_li.reverse()
    return(final_li)


# Finally, implement a Viterbi decoder that takes the model object and a dataset object and outputs tagged data, just like the tag_sentence function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

In [80]:
out_viterbi = viterbi_decoder(validation_dataset)
print(out_viterbi)

[[(Starting, 'O', 'O'), (from, 'O', 'O'), (the, 'B-DATE', 'O'), (23rd, 'I-DATE', 'O'), (,, 'O', 'O'), (the, 'O', 'O'), (Japanese, 'B-NORP', 'B-NORP'), (army, 'O', 'O'), (in, 'O', 'O'), (Yangquan, 'B-GPE', 'B-GPE'), (,, 'O', 'O'), (backed, 'O', 'O'), (by, 'O', 'O'), (helicopters, 'O', 'O'), (,, 'O', 'O'), (launched, 'O', 'O'), (nonstop, 'O', 'O'), (fierce, 'O', 'O'), (attacks, 'O', 'O'), (on, 'O', 'O'), (Shi'nao, 'B-LOC', 'B-CARDINAL'), (Mountain, 'I-LOC', 'B-PERSON'), (using, 'O', 'O'), (chemical, 'O', 'O'), (weapons, 'O', 'O'), (., 'O', 'O')]]


# Results from the Violation function on the output

In [81]:
print("Violations:",violations(out_viterbi))

Violations: 0


# Comparing the span-level metrics on baseline_output and your new output using your span_stats function

In [82]:
print("Span stats:",span_stats(out_viterbi))

Span stats: {'precision': 0.5, 'recall': 0.5, 'f1': 0.5}
